# In this file both trained models will translate first: Emoji to an english interpretation, Second: the emojis together with the english interpretation into a shakespearean english translation

In [8]:
import torch

emoji_to_sha_model_path = "./emoji-shakespeare-lora"
emoji_to_english_model_path = "./emoji-modern"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Few-shot examples
emoji_to_english_examples = [
    ("🐱📖", "The cat is reading a book."),
    ("🍎🍏", "An apple and another apple."),
    ("🌞💔😢🌈", "The sun is broken, I feel sad, but hope appears."),
]

english_to_sha_examples = [
    ("🐱📖", "The cat is reading a book.", "Behold the feline, engrossed in perusal of a tome."),
    ("🍎🍏", "An apple and another apple.", "A pome, and yet another pome, in mirthful harmony."),
    ("🌞💔😢🌈", "The sun is broken, I feel sad, but hope appears.",
     "Lo, the sun is rent asunder, mine heart doth ache, yet hope doth shine forth as a radiant bow."),
]

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

emoji2EnglishTokenizer = AutoTokenizer.from_pretrained(emoji_to_english_model_path)
emoji2EnglishModel = AutoModelForSeq2SeqLM.from_pretrained(emoji_to_english_model_path).to(device)
emoji2EnglishModel.eval()

emoji2ShaTokenizer = AutoTokenizer.from_pretrained(emoji_to_sha_model_path)
emoji2ShaModel = AutoModelForSeq2SeqLM.from_pretrained(emoji_to_sha_model_path).to(device)
emoji2ShaModel.eval()

Loading adapter weights from ./emoji-modern led to missing keys in the model: encoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight, encoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight, encoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight, encoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight, encoder.block.1.layer.0.SelfAttention.q.lora_A.default.weight, encoder.block.1.layer.0.SelfAttention.q.lora_B.default.weight, encoder.block.1.layer.0.SelfAttention.v.lora_A.default.weight, encoder.block.1.layer.0.SelfAttention.v.lora_B.default.weight, encoder.block.2.layer.0.SelfAttention.q.lora_A.default.weight, encoder.block.2.layer.0.SelfAttention.q.lora_B.default.weight, encoder.block.2.layer.0.SelfAttention.v.lora_A.default.weight, encoder.block.2.layer.0.SelfAttention.v.lora_B.default.weight, encoder.block.3.layer.0.SelfAttention.q.lora_A.default.weight, encoder.block.3.layer.0.SelfAttention.q.lora_B.default.weight, encoder.block.3.layer.0.SelfAttention.v.

T5ForConditionalGeneration(
  (shared): Embedding(384, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(384, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=768, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=768, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k): Linear(in_featu

In [7]:
def emoji_to_english(emoji_seq):
    prompt = "Translate emoji to an english sentence:\n"
    for e, eng in emoji_to_english_examples:
        prompt += f"Emoji: {e}\nModern English: {eng}\n\n"
    prompt += f"Emoji: {emoji_seq}\nModern English:"

    inputs = emoji2EnglishTokenizer(prompt, return_tensors="pt").to(device)
    outputs = emoji2EnglishModel.generate(**inputs, max_length=inputs['input_ids'].shape[1]+50)
    text = emoji2EnglishTokenizer.decode(outputs[0], skip_special_tokens=True)
    return text.split("Modern English:")[-1].strip()

def english_to_shakespeare(emoji_seq, english_text):
    prompt = "Translate the following into a Shakespearean English sentence, using both the emoji and Modern English as reference:\n"
    for e, eng, sha in english_to_sha_examples:
        prompt += f"Emoji: {e}\nModern English: {eng}\nShakespearean: {sha}\n\n"
    prompt += f"Emoji: {emoji_seq}\nModern English: {english_text}\nShakespearean:"

    inputs = emoji2ShaTokenizer(prompt, return_tensors="pt").to(device)
    outputs = emoji2ShaModel.generate(**inputs, max_length=inputs['input_ids'].shape[1]+100)
    text = emoji2ShaTokenizer.decode(outputs[0], skip_special_tokens=True)
    return text.split("Shakespearean:")[-1].strip()

def emoji_to_shakespeare(emoji_seq):
    english_translation = emoji_to_english(emoji_seq)
    shakespeare_translation = english_to_shakespeare(emoji_seq, english_translation)
    return shakespeare_translation
    torch.cuda.empty_cache()



In [9]:
import pandas as pd
from datasets import Dataset

df = pd.read_json("dataset.json")
dataset = Dataset.from_pandas(df)


In [13]:
import evaluate

sample_data = dataset.shuffle(seed=42).select(range(70))

bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")

preds = []
refs = []

for entry in sample_data:
    shakespeare = emoji_to_shakespeare(entry["emoji"])
    preds.append(shakespeare)
    refs.append([entry["shakespeare"]])

# Compute BLEU
bleu_result = bleu_metric.compute(predictions=preds, references=refs)
print(f"BLEU score: {bleu_result['score']:.2f}")

# Compute ROUGE
rouge_result = rouge_metric.compute(predictions=preds, references=[r[0] for r in refs])
print("ROUGE scores:")
for k, v in rouge_result.items():
    print(f"  {k}: {v:.4f}")

# Print sample results
print("\nSample predictions vs references:")
for emoji, pred, ref in zip([d["emoji"] for d in sample_data], preds, [r[0] for r in refs]):
    print(f"Emoji: {emoji}")
    print(f"Pred : {pred}")
    print(f"Ref  : {ref}")
    print("—" * 50)


BLEU score: 0.26
ROUGE scores:
  rouge1: 0.0721
  rouge2: 0.0006
  rougeL: 0.0709
  rougeLsum: 0.0714

Sample predictions vs references:
Emoji: 📚✏️📝🎓😴💤
Pred : A cat and I
Shake, I
Ref  : Though wisdom's crown awaits, these weary eyes grow heavy o'er my learned tomes and slumber calls.
——————————————————————————————————————————————————
Emoji: 🏝️😤☕🥞🚢
Pred : I am so sorry that I am so sorry that I am so sorry that I am so sorry that I am so sorry that I am so sorry that I am so sorry that I am so so so so so so so so
Ref  : Upon a deserted isle, a scorned soul doth seethe, seeking solace in a cup of bitter brew, whilst dreaming of fluffy breakfast banquet aboard a noble vessel.
——————————————————————————————————————————————————
Emoji: 🌸🐝☀️😊
Pred : sentence is a shakespearean translation of the sentence and the emoji is a shakespearean translation.
The sentence is a shakespearean translation and the emoji is a shakespearean e.
S. "
Ref  : Sweet workers dance 'round blooms whilst Phoebus sm